In [2]:
import pandas as pd
import numpy as np

In [27]:


# 1. Forecast Anual de pedidos por año (2025-2029)
forecast_anual = {
    2025: 12000,
    2026: 13500,
    2027: 15000,
    2028: 16500,
    2029: 18000
}

# 2. Distribución histórica de pedidos por tienda
distribucion_tienda = pd.Series([0.30, 0.25, 0.20, 0.15, 0.10], index=[f'Tienda {i+1}' for i in range(5)])
distribucion_tienda /= distribucion_tienda.sum()

# 3. Distribución mensual histórica por tienda
distribucion_mensual = pd.DataFrame([
    [0.08, 0.07, 0.06, 0.08, 0.09, 0.10, 0.09, 0.07, 0.06, 0.08, 0.07, 0.05],
    [0.07, 0.06, 0.05, 0.07, 0.08, 0.09, 0.08, 0.06, 0.05, 0.07, 0.06, 0.04],
    [0.10, 0.09, 0.08, 0.10, 0.11, 0.12, 0.11, 0.09, 0.08, 0.10, 0.09, 0.07],
    [0.05, 0.04, 0.03, 0.05, 0.06, 0.07, 0.06, 0.04, 0.03, 0.05, 0.04, 0.03],
    [0.04, 0.03, 0.02, 0.04, 0.05, 0.06, 0.05, 0.03, 0.02, 0.04, 0.03, 0.02]
], index=[f'Tienda {i+1}' for i in range(5)], columns=[f'Mes {i+1}' for i in range(12)])

distribucion_mensual = distribucion_mensual.div(distribucion_mensual.sum(axis=1), axis=0)

# 4. Generar distribuciones dinámicas por año
años = list(forecast_anual.keys())

distribucion_tipo_pedido = pd.DataFrame({
    year: [0.70 - 0.05 * (year - 2025), 0.30 + 0.05 * (year - 2025)] for year in años
}, index=["Parcel", "Big Item"])

distribucion_categoria_pedido = pd.DataFrame({
    year: [0.20, 0.20, 0.20, 0.20, 0.20] for year in años
}, index=["Cat 1", "Cat 2", "Cat 3", "Cat 4", "Cat 5"])

distribucion_basket_size = pd.DataFrame({
    year: [0.25, 0.25, 0.25, 0.25] for year in años
}, index=["0-5", "6-10", "11-20", "+20"])

distribucion_squad = pd.DataFrame({
    year: [0.25, 0.30, 0.20, 0.25] for year in años
}, index=["Squad A", "Squad B", "Squad C", "Squad D"])

# 5. Función para aplicar Iterative Proportional Fitting (IPF)
def ipf(matriz, objetivo_filas, objetivo_columnas, iteraciones=100, tolerancia=1e-5):
    for _ in range(iteraciones):
        matriz *= (objetivo_filas[:, None] / matriz.sum(axis=1, keepdims=True))
        matriz *= (objetivo_columnas / matriz.sum(axis=0, keepdims=True))
    return matriz

# 6. Aplicar IPF con cambios de proporciones en el tiempo
def ajustar_pedidos_ipf(forecast_anual, distribucion_tienda, distribucion_mensual, tipo_pedido, categoria_pedido, basket_size, squad, ajustes_tipo_pedido={}):
    pedidos_resultantes = {}
    
    for year, total_pedidos in forecast_anual.items():
        pedidos_por_tienda = total_pedidos * distribucion_tienda
        matriz_pedidos = pedidos_por_tienda.values[:, None] * distribucion_mensual.values
        
        # Ajuste con IPF
        objetivo_filas = pedidos_por_tienda.values
        objetivo_columnas = matriz_pedidos.sum(axis=0)
        matriz_pedidos = ipf(matriz_pedidos, objetivo_filas, objetivo_columnas)
        
        for tipo in tipo_pedido.index:
            for categoria in categoria_pedido.index:
                for size in basket_size.index:
                    for squad_name in squad.index:
                        matriz_ajustada = matriz_pedidos * tipo_pedido.loc[tipo, year] * categoria_pedido.loc[categoria, year] * basket_size.loc[size, year] * squad.loc[squad_name, year]
                        pedidos_resultantes[(year, tipo, categoria, size, squad_name)] = matriz_ajustada
    
    return pedidos_resultantes

# 7. Exportar resultados a Excel
def exportar_resultados_excel(resultados, nombre_archivo="/Users/rock/Documents/Back_end/resultados_ipf.xlsx"):
    with pd.ExcelWriter(nombre_archivo) as writer:
        for year in forecast_anual.keys():
            df_lista = []
            for key, matriz in resultados.items():
                if key[0] == year:
                    df = pd.DataFrame(matriz, index=distribucion_tienda.index, columns=[f'Mes {i+1}' for i in range(12)])
                    df["Tipo"] = key[1]
                    df["Categoría"] = key[2]
                    df["Basket Size"] = key[3]
                    df["Squad"] = key[4]
                    df_lista.append(df)
            df_final = pd.concat(df_lista)
            df_final.to_excel(writer, sheet_name=str(year))

# 8. Ejecutar el modelo y exportar resultados
resultados = ajustar_pedidos_ipf(
    forecast_anual, distribucion_tienda, distribucion_mensual,
    distribucion_tipo_pedido, distribucion_categoria_pedido, distribucion_basket_size,
    distribucion_squad
)
exportar_resultados_excel(resultados)